# Importando as bibliotecas necessárias

In [13]:
from math import*
import numpy as np

# Declarando as variáveis necessárias

In [14]:
m = 3 #número de nós
n = 3 #número de elementos
E = 210e9 #módulo de elasticidade(Constante)
matriz_N = np.zeros([2,n]) #matriz dos nós
matriz_C = np.zeros([n,m]) #matriz conectividade
matriz_A = np.ones([m,1]) * 2e-4 #área da secção transversal (m²)
matriz_F = np.zeros([2*n,1]) 

# Adicionando a medida de cada nó

In [15]:
matriz_N[0,:] = [0,0,0.3]
matriz_N[1,:] = [0,0.4,0.4]

# Adicionando cada termo da conectividade 

### 0 = nó independente
### 1 = nó final
### -1 = nó inicial

In [16]:
matriz_C[0,0] = -1 
matriz_C[1,0] = 1

matriz_C[1,1] = -1
matriz_C[2,1] = 1

matriz_C[0,2] = -1
matriz_C[2,2] = 1

print(matriz_C)

[[-1.  0. -1.]
 [ 1. -1.  0.]
 [ 0.  1.  1.]]


In [17]:
matriz_membros = matriz_N @ matriz_C #criação das matriz dos membros
print(matriz_membros)

[[0.  0.3 0.3]
 [0.4 0.  0.4]]


# Formação de matriz_S

In [18]:
lista_L = []
lista_matriz_S = []
    
for i in range(m):
    
    L = np.linalg.norm(matriz_membros[:,i])
    lista_L.append(L)
    c = matriz_membros[:,i]
    c.shape = [2,1]
    matriz_S = ((E * matriz_A[i,0])/L) * ((c) @ (c.T)) / (L**2)
    lista_matriz_S.append(matriz_S)
    
    print(matriz_S)

[[0.00e+00 0.00e+00]
 [0.00e+00 1.05e+08]]
[[1.4e+08 0.0e+00]
 [0.0e+00 0.0e+00]]
[[30240000.         40320000.        ]
 [40320000.         53760000.00000001]]


# Formação matriz de rigidez global

In [19]:
tamanho_S = len(matriz_S)
tamanho_C = len(matriz_C)
matriz_G = np.zeros([tamanho_S * tamanho_C,tamanho_S * tamanho_C])

for j in range(m):

    t = matriz_C[:,j]
    t.shape = [3,1]
    
    K = np.kron(((t) @ (t.T)), lista_matriz_S[j])
    matriz_G += K
    
print(matriz_G)
matriz_KG = matriz_G

determinante = np.linalg.det(matriz_G)

[[ 3.0240e+07  4.0320e+07  0.0000e+00  0.0000e+00 -3.0240e+07 -4.0320e+07]
 [ 4.0320e+07  1.5876e+08  0.0000e+00 -1.0500e+08 -4.0320e+07 -5.3760e+07]
 [ 0.0000e+00  0.0000e+00  1.4000e+08  0.0000e+00 -1.4000e+08  0.0000e+00]
 [ 0.0000e+00 -1.0500e+08  0.0000e+00  1.0500e+08  0.0000e+00  0.0000e+00]
 [-3.0240e+07 -4.0320e+07 -1.4000e+08  0.0000e+00  1.7024e+08  4.0320e+07]
 [-4.0320e+07 -5.3760e+07  0.0000e+00  0.0000e+00  4.0320e+07  5.3760e+07]]


In [20]:
import numpy as np

# Definindo o vetor de constantes (F)
R1X = None
R1Y = 0

R2X = None
R2Y = None

R3X = 150
R3Y = -100
# criando vetor de constantes sem as desconhecidas
Lista_R=[R1X, R1Y, R2X, R2Y, R3X, R3Y]
Listafinal_R=[]
for val in Lista_R:
    if val != None:
        Listafinal_R.append(val)

newF = np.array(Listafinal_R)
newF.shape = (len(Listafinal_R), 1)
        
F = np.array([R1X, R1Y, R2X, R2Y, R3X, R3Y])
F.shape = (6, 1)
# Definindo as matrizes A e B de exemplo

# Encontrando índices onde A tem elementos None
linhas_a, colunas_a = np.where(F == None)
print(linhas_a,colunas_a)

# Removendo as linhas e colunas correspondentes na matriz B
i=0
for linha, coluna in zip(linhas_a, colunas_a):
    print(linha)
    matriz_G = np.delete(matriz_G, linha-i, axis=0)  # Removendo a linha
    matriz_G = np.delete(matriz_G, linha-i, axis=1)  # Removendo a coluna
    i+=1
    

print("\nMatriz G após a remoção das linhas e colunas correspondentes:")
print(matriz_G)


[0 2 3] [0 0 0]
0
2
3

Matriz G após a remoção das linhas e colunas correspondentes:
[[ 1.5876e+08 -4.0320e+07 -5.3760e+07]
 [-4.0320e+07  1.7024e+08  4.0320e+07]
 [-5.3760e+07  4.0320e+07  5.3760e+07]]


# Declarar vetor global de forças concentradas

In [21]:
# Resolvendo o sistema linear Ax = B
    
u = np.linalg.solve(matriz_G, newF)

print("\nA solução do sistema é:")
print(u)



A solução do sistema é:
[[-9.52380952e-07]
 [ 1.60714286e-06]
 [-4.01785714e-06]]


In [22]:
u_original = np.zeros([2*n, 1])

j=0
for i in range(len(u_original)):
    if i not in linhas_a:
        u_original[i] = u[j]
        j+=1
        


    

print(u_original)

[[ 0.00000000e+00]
 [-9.52380952e-07]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.60714286e-06]
 [-4.01785714e-06]]


<h2>Forças em cada grau de liberdade</h2>

In [23]:
f = matriz_KG @ u_original 
print(f)

[[  75.]
 [   0.]
 [-225.]
 [ 100.]
 [ 150.]
 [-100.]]


In [24]:
tens_nos_elementos=[]
for i in range(n):
    u_elemento = np.zeros((4,1))
    coluna = matriz_C[:,i]
    j=0
    for n in coluna:
        if n== -1:
            conj1 = j
        if n==1: 
            conj2 =j
        j+=1
    conjs = [conj1,conj2]
    cont= 0
    x1= matriz_N[0][conj1]
    x2 = matriz_N[0][conj2]
    y1= matriz_N[1][conj1]
    y2 = matriz_N[1][conj2]
    for conj in conjs:
        u_elemento[(cont*2)] = u_original[conj*2]
        u_elemento[(cont*2)+1] = u_original[(conj*2)+1]
        cont+=1
    print(u_elemento)
    cscs = np.zeros((1,4))
    co= (x2 -x1)/lista_L[i]
    se= (y2 -y1)/lista_L[i]
    cscs[0]=[-co,-se,co,se]
    _E_=210e9
    tens = (_E_/lista_L[i])*(cscs @ u_elemento)
    print("------")
    print(tens)
    print("------")
    tens_nos_elementos.append(tens)
    

        



    
            



[[ 0.00000000e+00]
 [-9.52380952e-07]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
------
[[500000.]]
------
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.60714286e-06]
 [-4.01785714e-06]]
------
[[1125000.]]
------
[[ 0.00000000e+00]
 [-9.52380952e-07]
 [ 1.60714286e-06]
 [-4.01785714e-06]]
------
[[-625000.]]
------
